In [0]:
import pandas as pd
import googlemaps
from itertools import tee

In [0]:
# Read the file with postcode - Fine out travel journey from these postcodes
data = pd.read_csv('../../postcode_data/pcd_data_watford.csv')
data = data.drop(columns={'Unnamed: 0'})

In [0]:
print('Number Postcodes: ' + str(len(data)))

In [0]:
# All Wembley Data for weekendday (1)
data_weekends = data[data['weekend_day']==1].reset_index(drop=True)
# Weekend Day Midday Time (0)
data_weekends0 = data_weekends[data_weekends['timeofday_id'] == 0].reset_index(drop=True)
# Weekend Day Afternoon Time (1)
data_weekends1 = data_weekends[data_weekends['timeofday_id'] == 1].reset_index(drop=True)
# Weekend Day Evening Time (2)
data_weekends2 = data_weekends[data_weekends['timeofday_id'] == 2].reset_index(drop=True)

In [0]:
# All Wembley Data for weekday (0)
data_week = data[data['weekend_day']==0].reset_index(drop=True)
# Weekend Day Midday Time (0)
data_week0 = data_week[data_week['timeofday_id'] == 0].reset_index(drop=True)
# Weekend Day Afternoon (1)
data_week1 = data_week[data_week['timeofday_id'] == 1].reset_index(drop=True)
# Weekend Day Evening Time (2)
data_week2 = data_week[data_week['timeofday_id'] == 2].reset_index(drop=True)

In [0]:
print('Weekends Midday: ' + str(len(data_weekends0)))
print('Weekends Afternoon: ' + str(len(data_weekends1)))
print('Weekends Evening: ' + str(len(data_weekends2)))
print(' ')
print('Week Midday: ' + str(len(data_week0)))
print('Week Afternoon: ' + str(len(data_week1)))
print('Week Evening: ' + str(len(data_week2)))

Weekends Midday: 0
Weekends Afternoon: 584
Weekends Evening: 0
 
Week Midday: 0
Week Afternoon: 123
Week Evening: 157


In [0]:
data_weekends1.tail(2)

In [0]:
# Read the Destination File - Stadium Centroid Location
stadium = pd.read_csv('postcode_data/data_stadium_centroid.csv')
stadium = stadium.drop(columns={'Unnamed: 0'})

In [0]:
stadium.tail(6)

,venue,lon,lat
0,london_stadium,-0.016519,51.538714
1,wembley,-0.279321,51.556244
2,twickenham,-0.341428,51.455873
3,wembley_arena,-0.282766,51.558053
4,vicarage_road,-0.401386,51.649834


In [0]:
# Google Maps API Key
API_key = 'Your API Key'
gmaps = googlemaps.Client(key=API_key)

### Run the Google API Distance-Time Matrix
According to the time of the event (midday, afternoon, evening) and the day of the event (weekdays, weekends) adapt the arrival time. 

1. Weekends Midday : Saturday, Arrival 12:00 BST/11am UCT - Timestamp: 1564830000 / 1565434800
2. Weekends Afternoon : Saturday, Arrival 2pm BST/1pm UCT - Timestamp: 1564837200 / 1565442000
3. Weekends Evening : Saturday, Arrival 6pm BST/5pm UCT - Timestamp: 1564851600 / 1565456400


4. Weekends Midday : Friday, Arrival 1pm BST/12:00 UCT - Timestamp: 1564747200 
5. Weekends Afternoon : Friday, Arrival 2pm BST/1pm UCT - Timestamp: 1564750800 / 1565352000
6. Weekends Evening : Friday, Arrival 6pm BST/5pm UCT - Timestamp: 1564765200 / 1565370000

In [0]:
## GOOGLE DISTANCE - TIME MATRIX API 

# set destination lat/lon
dest_wembley = (stadium[stadium['venue']=='vicarage_road'].lat, stadium[stadium['venue']=='vicarage_road'].lon)

# empty list to store the distance
distance_list = []
time_list = []
for i, row in data_week2.iterrows(): 
    
    # set origin lat/lon
    origin_lat = row['lat']
    origin_lon = row['lon']
    origins = (origin_lat, origin_lon)
    
    try: 
        result_distance = gmaps.distance_matrix(origins, dest_wembley, arrival_time=1565370000, mode='driving')["rows"][0]["elements"][0]["distance"]["value"]
    except:
        result_distance = 0
        
    try: 
        result_time = gmaps.distance_matrix(origins, dest_wembley, arrival_time=1565370000, mode='driving')["rows"][0]["elements"][0]["duration"]["value"]
    except:
        result_time = 'not found'
        
    #calculate distance in km
    result_km = result_distance/1000

    #append result to list
    distance_list.append(result_km)
    time_list.append(result_time)
    

In [0]:
print('Length Distance list: ' + str(len(distance_list)))
print('Length Time list: ' + str(len(time_list)))

Length Distance list: 157
Length Time list: 157


In [0]:
data_week2['driving_dist_km'] = distance_list
data_week2['driving_time_min'] = time_list

In [0]:
data_week2.head(2)

In [0]:
data_week2.tail(2)

In [0]:
data_week2.to_csv('data_week2_watford.csv')

In [0]:
data3 = pd.read_csv('data_week1_watford.csv')
data4 = pd.read_csv('data_weekends1_watford.csv')
data5 = pd.read_csv('data_week2_watford.csv')

In [0]:
data_list = [data3,data4]
data_watford = data5.append(data_list)

In [0]:
len(data_watford)

864

In [0]:
data_watford = data_watford.drop(columns={'Unnamed: 0'})
data_watford = data_watford.reset_index(drop=True)
#data_olympic = data_olympic.rename(columns={'transit_dist_km':'driving_dist_km','transit_time_min':'driving_time_min'})
data_watford.head(2)

In [0]:
data_watford.tail(2)

In [0]:
data_watford.to_csv('data_watford_driving.csv')

In [0]:
## Merge All Stadiums together! (Wembley, Olympic, Twickenham, Watford)
# 1. Read all the dataframe
data_wembley = pd.read_csv('data_wembley_driving.csv')
data_watford = pd.read_csv('data_watford_driving.csv')
data_twickenham = pd.read_csv('data_twickenham_driving.csv')
data_olympic = pd.read_csv('data_olympic_driving.csv')

In [0]:
print('Wembley: ' + str(len(data_wembley)))
print('Watford: ' + str(len(data_watford)))
print('Twickenham: ' + str(len(data_twickenham)))
print('London Olympic: ' + str(len(data_olympic)))

Wembley: 14399
Watford: 864
Twickenham: 3851
London Olympic: 3530


In [0]:
# Need to Add the venueid for each Dataset!
data_wembley['venueid'] = 4
data_watford['venueid'] = 2
data_twickenham['venueid'] = 1
data_olympic['venueid'] = 3

In [0]:
# Append dataframes
data_stadium_list = [data_wembley,data_watford,data_twickenham]
data_stadium = data_olympic.append(data_stadium_list)

# Drop Unnamed columns and index
data_stadium = data_stadium.drop(columns={'Unnamed: 0'})
data_stadium = data_stadium.reset_index(drop=True)
data_stadium.head(2)

In [0]:
len(data_stadium)

22644

In [0]:
print('Undefined Travel distance - Data Number: ' + str(len(data_stadium[data_stadium['driving_dist_km']==0])))

Undefined Travel distance - Data Number: 0


In [0]:
data_stadium.to_csv('googleAPI_distance&time_driving_travel.csv')